In [1]:
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append("../..")
from naslib.utils import get_dataset_api

from naslib.search_spaces.core import Metric


# NASBench201

In [2]:
from naslib.search_spaces import nasbench201 as nb201
from naslib.search_spaces.nasbench201.graph import NasBench201SearchSpace

from nas_201_api import NASBench201API

cifar10_dataset_api = get_dataset_api("nasbench201", 'cifar10')
cifar100_dataset_api = get_dataset_api("nasbench201", 'cifar100')
real_nb201_api = NASBench201API("../../naslib/data/NAS-Bench-201-v1_0-e61699.pth", verbose=False)
from xautodl.models import get_cell_based_tiny_net
from naslib.search_spaces.nasbench201.conversions import convert_str_to_op_indices, convert_op_indices_to_naslib


0it [00:00, ?it/s]


KeyError: 'cifar100'

In [13]:
import numpy as np

df = pd.DataFrame(columns=["arch_id", "arch_str", "params", "cifar_10_val_accuracy"])

for i, arch_str in tqdm(enumerate(real_nb201_api)):
    indices = convert_str_to_op_indices(arch_str)
    objec = NasBench201SearchSpace()
    convert_op_indices_to_naslib(indices, objec)
    cifar_10_val_accuracy = objec.query(dataset='cifar10', metric=Metric.VAL_ACCURACY, dataset_api=cifar10_dataset_api)
    cifar_100_val_accuracy = objec.query(dataset='cifar100', metric=Metric.VAL_ACCURACY, dataset_api=cifar100_dataset_api)

    id = real_nb201_api.query_index_by_arch(arch_str)
    config = real_nb201_api.get_net_config(id, 'cifar10-valid')  # obtain the network configuration for the n-th architecture on the CIFAR-10 dataset
    network = get_cell_based_tiny_net(config)  # create the network from configuration
    params = sum([np.prod(p.size()) for p in network.parameters()])

    df = df.append({"arch_id": id, "arch_str": arch_str, "params": params, "cifar_10_val_accuracy": cifar_10_val_accuracy}, ignore_index=True)

15625it [52:34,  4.95it/s] 


In [14]:
df.to_csv("nasbench201.csv")

In [15]:
df.head()

,arch_id,arch_str,params,cifar_10_val_accuracy
0,0,|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~...,129306,81.90
1,1,|nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~...,802426,90.39
2,2,|avg_pool_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~...,559386,81.93
3,3,|avg_pool_3x3~0|+|skip_connect~0|none~1|+|none...,73306,83.78
4,4,|skip_connect~0|+|skip_connect~0|nor_conv_1x1~...,129306,84.26


# NAS-Bench-101

In [10]:
from naslib.search_spaces import nasbench101 as nb101
from nasbench import api
nasbench101_api = api.NASBench('../../naslib/data/nasbench_full.tfrecord')


Loading dataset from file... This may take a few minutes...
Loaded dataset in 74 seconds


NameError: name 'get_dataset_api' is not defined

In [24]:
from naslib.utils import get_dataset_api

nb101_api = get_dataset_api("nasbench101", 'cifar10')


Loading dataset from file... This may take a few minutes...
Loaded dataset in 3 seconds


In [25]:
nb101_data = nb101_api["nb101_data"]

In [45]:
df = pd.DataFrame(columns=["arch_id", "arch_hash", "params", "cifar_10_val_accuracy"])
hash_iterator = nb101_data.hash_iterator()
for i, hash in tqdm(enumerate(hash_iterator), total=len(hash_iterator)):
    metrics = nb101_data.get_metrics_from_hash(hash)
    params = metrics[0]["trainable_parameters"]
    cifar_10_val_accuracy = metrics[1][108][0]["final_validation_accuracy"]
    df = df.append({"arch_id": i, "arch_hash": hash, "params": params, "cifar_10_val_accuracy": cifar_10_val_accuracy}, ignore_index=True)

100%|██████████| 423624/423624 [1:08:46<00:00, 102.67it/s]


In [46]:
df.to_csv("nasbench101.csv")

# NATS-Bench

In [2]:
from naslib.search_spaces import natsbenchsize as nbs
dataset_api = get_dataset_api("natsbenchsize", 'cifar100')

cifar10_dataset_api = get_dataset_api("natsbenchsize", 'cifar100')
df = pd.DataFrame(columns=["arch_id", "arch_channels", "params", "cifar_100_val_accuracy"])


[2024-12-05 10:12:51] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
[2024-12-05 10:12:51] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.


In [ ]:
import nats_bench
import xautodl
from xautodl.models import get_cell_based_tiny_net
real_api = nats_bench.create("/home/lam/projets/nas_ntk/naslib/data/NATS-sss-v1_0-50262-simple", "sss", fast_mode=True, verbose=False)
for i, arch in tqdm(enumerate(dataset_api), total=len(dataset_api)):
    a = nbs.graph.NATSBenchSizeSearchSpace()
    channels = [int(e) for e in arch.split(":")]
    # 2. Find the associated architecture index in the api
    index = real_api.query_index_by_arch(arch)
    # 3. Fetch desired metric from API.
    config = real_api.get_net_config(index, 'cifar10')
    network = get_cell_based_tiny_net(config)
    n_params = sum(p.numel() for p in network.parameters())
    a.set_channels(channels)
    cifar_100_val_accuracy = a.query(dataset='cifar100', metric=Metric.VAL_ACCURACY, dataset_api=cifar10_dataset_api)
    #df = df.append({"arch_id": i, "arch_channels": arch, "params": n_params, "cifar_100_val_accuracy": cifar_100_val_accuracy}, ignore_index=True)

 54%|█████████████████████████████████████████████████▊                                          | 17757/32768 [03:41<03:06, 80.54it/s]

In [ ]:
df.to_csv("natsbenchsize.csv")